In [ ]:
# !pip install keras
# !pip install tensorflow
# !pip install numpy
# !pip install pandas
from keras.models import Model
from keras.layers import Dropout, Dense, Input, LSTM, concatenate, ConvLSTM2D, Conv2D, Lambda, Reshape
from keras.losses import Huber
from keras.optimizers import Adam
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.nn import softmax, leaky_relu
from tensorflow import expand_dims, einsum
import tensorflow as tf
from datetime import datetime as dt

# Utils

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

index_Spine_Base = 0
index_Spine_Mid = 4
index_Neck = 8
index_Head = 12  # no orientation
index_Shoulder_Left = 16
index_Elbow_Left = 20
index_Wrist_Left = 24
index_Hand_Left = 28
index_Shoulder_Right = 32
index_Elbow_Right = 36
index_Wrist_Right = 40
index_Hand_Right = 44
index_Hip_Left = 48
index_Knee_Left = 52
index_Ankle_Left = 56
index_Foot_Left = 60  # no orientation
index_Hip_Right = 64
index_Knee_Right = 68
index_Ankle_Right = 72
index_Foot_Right = 76  # no orientation
index_Spine_Shoulder = 80
index_Tip_Left = 84  # no orientation
index_Thumb_Left = 88  # no orientation
index_Tip_Right = 92  # no orientation
index_Thumb_Right = 96  # no orientation


class Data_Loader:
    def __init__(self, dir):
        self.num_repitation = 5
        self.num_channel = 3
        self.dir = dir
        self.body_part = self.body_parts()
        self.dataset = []
        self.sequence_length = []
        self.num_timestep = 100
        self.new_label = []
        self.train_x, self.train_y = self.import_dataset()
        self.batch_size = self.train_y.shape[0]
        self.num_joints = len(self.body_part)
        self.sc1 = StandardScaler()
        self.sc2 = StandardScaler()
        self.scaled_x, self.scaled_y = self.preprocessing()

    def body_parts(self):
        body_parts = [
            index_Spine_Base,
            index_Spine_Mid,
            index_Neck,
            index_Head,
            index_Shoulder_Left,
            index_Elbow_Left,
            index_Wrist_Left,
            index_Hand_Left,
            index_Shoulder_Right,
            index_Elbow_Right,
            index_Wrist_Right,
            index_Hand_Right,
            index_Hip_Left,
            index_Knee_Left,
            index_Ankle_Left,
            index_Foot_Left,
            index_Hip_Right,
            index_Knee_Right,
            index_Ankle_Right,
            index_Ankle_Right,
            index_Spine_Shoulder,
            index_Tip_Left,
            index_Thumb_Left,
            index_Tip_Right,
            index_Thumb_Right,
        ]
        return body_parts

    def import_dataset(self):
        train_x = (
            pd.read_csv("./" + self.dir + "/Train_X.csv", header=None).iloc[:, :].values
        )
        train_y = (
            pd.read_csv("./" + self.dir + "/Train_Y.csv", header=None).iloc[:, :].values
        )
        return train_x, train_y

    def preprocessing(self):
        X_train = np.zeros(
            (self.train_x.shape[0], self.num_joints * self.num_channel)
        ).astype("float32")
        for row in range(self.train_x.shape[0]):
            counter = 0
            for parts in self.body_part:
                for i in range(self.num_channel):
                    X_train[row, counter + i] = self.train_x[row, parts + i]
                counter += self.num_channel

        y_train = np.reshape(self.train_y, (-1, 1))
        X_train = self.sc1.fit_transform(X_train)
        y_train = self.sc2.fit_transform(y_train)

        X_train_ = np.zeros(
            (self.batch_size, self.num_timestep, self.num_joints, self.num_channel)
        )

        for batch in range(X_train_.shape[0]):
            for timestep in range(X_train_.shape[1]):
                for node in range(X_train_.shape[2]):
                    for channel in range(X_train_.shape[3]):
                        X_train_[batch, timestep, node, channel] = X_train[
                            timestep + (batch * self.num_timestep),
                            channel + (node * self.num_channel),
                        ]

        X_train = X_train_
        return X_train, y_train

In [ ]:
import tensorflow as tf
import numpy as np

class Graph():
    def __init__(self, num_node):
        self.num_node = num_node
        self.AD, self.AD2, self.bias_mat_1, self.bias_mat_2 = self.normalize_adjacency()
        
    def normalize_adjacency(self):
        self_link = [(i, i) for i in range(self.num_node)]
        neighbor_1base = [(1, 2), (2, 21), (3, 21), (4, 3), (5, 21),
                                      (6, 5), (7, 6), (8, 7), (9, 21), (10, 9),
                                      (11, 10), (12, 11), (13, 1), (14, 13), (15, 14),
                                      (16, 15), (17, 1), (18, 17), (19, 18), (20, 19),
                                      (22, 23), (23, 8), (24, 25), (25, 12)]
        neighbor_link = [(i - 1, j - 1) for (i, j) in neighbor_1base]
        edge = self_link + neighbor_link    
        A = np.zeros((self.num_node, self.num_node)) # adjacency matrix
        for i, j in edge:
            A[j, i] = 1
            A[i, j] = 1
        
        A2 = np.zeros((self.num_node, self.num_node)) # second order adjacency matrix
        for root in range(A.shape[1]):
            for neighbour in range(A.shape[0]):
                if A[root, neighbour] == 1:
                    for neighbour_of_neigbour in range(A.shape[0]):
                        if A[neighbour, neighbour_of_neigbour] == 1:
                            A2[root,neighbour_of_neigbour] = 1                 
        #AD = self.normalize(A)
        #AD2 = self.normalize(A2)
        bias_mat_1 = np.zeros(A.shape)
        bias_mat_2 = np.zeros(A2.shape)
        bias_mat_1 = np.where(A!=0, bias_mat_1, -1e9)
        bias_mat_2 = np.where(A2!=0, A2, -1e9)
        AD = A.astype('float32')
        AD2 = A2.astype('float32')
        bias_mat_1 = bias_mat_1.astype('float32')
        bias_mat_2 = bias_mat_2.astype('float32')
        AD = tf.convert_to_tensor(AD)
        AD2= tf.convert_to_tensor(AD2)
        bias_mat_1 = tf.convert_to_tensor(bias_mat_1)
        bias_mat_2 = tf.convert_to_tensor(bias_mat_2)
        return AD, AD2, bias_mat_1, bias_mat_2
        
    def normalize(self, adjacency):
        rowsum = np.array(adjacency.sum(1))
        r_inv = np.power(rowsum, -1).flatten()
        r_inv[np.isinf(r_inv)] = 0
        r_mat_inv = np.diag(r_inv)
        normalize_adj = r_mat_inv.dot(adjacency)
        normalize_adj = normalize_adj.astype('float32')
        normalize_adj = tf.convert_to_tensor(normalize_adj)   
        return normalize_adj

# Model

In [16]:
class Sgcn_Lstm():
    def __init__(self, train_x, train_y, valid_x, valid_y, AD, AD2, lr=0.0001, epoach=200, batch_size=10):
        self.train_x = train_x
        self.train_y = train_y
        self.valid_x = valid_x
        self.valid_y = valid_y
        self.AD = AD
        self.AD2 = AD2
        self.lr = lr
        self.epoach =epoach
        self.batch_size = batch_size
        self.num_joints = 25

    def sgcn(self, Input):

        # number_of_filters = 64

        x = tf.keras.layers.Conv2D(filters=64, kernel_size=(1,1), strides=1, activation='relu')(Input)
        x = Dropout(0.25)(x)

        gcn_1 = tf.keras.layers.Lambda(lambda x: tf.einsum('vw,ntwc->ntvc', x[0], x[1]))([self.AD, x])

        # gcn_1 = tf.keras.layers.Lambda(
        #     lambda x: tf.einsum('vw,ntwc->ntvc', x[0], x[1]),
        #     output_shape=(x.shape[0], x.shape[1], x.shape[2], number_of_filters)  # Specify number_of_filters
        #     )([self.AD, x])

        y = tf.keras.layers.Conv2D(filters=64, kernel_size=(1,1), strides=1, activation='relu')(Input)
        y = Dropout(0.25)(y)
        gcn_2 = tf.keras.layers.Lambda(lambda x: tf.einsum('vw,ntwc->ntvc', x[0], x[1]))([self.AD2, y])
        gcn = concatenate([gcn_1, gcn_2], axis=-1)                                                                                                                                   
        
        x = tf.keras.layers.Conv2D(filters=128, kernel_size=(1,1), strides=1, activation='relu')(gcn)
        x = Dropout(0.25)(x)
        gcn_1 = tf.keras.layers.Lambda(lambda x: tf.einsum('vw,ntwc->ntvc', x[0], x[1]))([self.AD, x])
        y = tf.keras.layers.Conv2D(filters=128, kernel_size=(1,1), strides=1, activation='relu')(gcn)
        y = Dropout(0.25)(y)
        gcn_2 = tf.keras.layers.Lambda(lambda x: tf.einsum('vw,ntwc->ntvc', x[0], x[1]))([self.AD2, y])
        gcn = concatenate([gcn_1, gcn_2], axis=-1)
        
        gcn = tf.keras.layers.Reshape(target_shape=(-1,gcn.shape[2]*gcn.shape[3]))(gcn)
        return gcn

    def Lstm(self,x):
        rec = LSTM(80, return_sequences=True)(x)
        rec = Dropout(0.25)(rec)
        rec1 = LSTM(40, return_sequences=True)(rec)
        rec1 = Dropout(0.25)(rec1)
        rec2 = LSTM(40, return_sequences=True)(rec1)
        rec2 = Dropout(0.25)(rec2)
        rec3 = LSTM(80)(rec2)
        rec3 = Dropout(0.25)(rec3)
        output = Dense(1, activation = 'linear')(rec3)
        return output

    def build(self):
        seq_input = Input(shape=(None, self.train_x.shape[2], self.train_x.shape[3]), batch_size=None)
        sgcn_layer = self.sgcn(seq_input)
        lstm_sgcn_layer = self.Lstm(sgcn_layer)
        self.model = Model(seq_input, lstm_sgcn_layer)
        return self.model     

    def train(self):
        t = dt.now()

        self.model.compile(loss=tf.keras.losses.Huber(delta=0.1), optimizer= Adam(learning_rate=self.lr))

        early_stopping = EarlyStopping(monitor='val_loss', patience = 25)
        checkpoint = ModelCheckpoint("models/model_ex5.keras", monitor='val_loss', save_best_only=True, mode='auto', save_freq='epoch')
        
        history = self.model.fit(self.train_x, self.train_y, validation_data = (self.valid_x,self.valid_y), epochs=self.epoach, batch_size=self.batch_size, callbacks=[checkpoint, early_stopping])

        print('Training time: %s' % (dt.now() - t))

        self.model.save_weights("models/my_model_trained_exercise.h5")

        return history
    
    def prediction(self, data):
        y_pred = self.model.predict(data)
        return y_pred

# Train Begin

In [17]:
import numpy as np

from sklearn.model_selection import train_test_split
from keras.optimizers import *
from sklearn.model_selection import train_test_split

from math import sqrt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

# from data_processing import Data_Loader
# from graph import Graph
# from GCN.sgcn_lstm import Sgcn_Lstm

from sklearn.metrics import mean_squared_error, mean_absolute_error

random_seed = 42  # for reproducibility


In [18]:
exercise = 'data/KIMORE/Kimore_ex5'
learning_rate = 0.0001
epoch = 500
batch_size = 10

"""import the whole dataset"""
data_loader = Data_Loader(exercise)  # folder name -> Train.csv, Test.csv

"""import the graph data structure"""
graph = Graph(len(data_loader.body_part))

"""Split the data into training and validation sets while preserving the distribution"""
train_x, valid_x, train_y, valid_y = train_test_split(data_loader.scaled_x, data_loader.scaled_y, test_size=0.2, random_state = random_seed)

print("Training instances: ", len(train_x))
print("Validation instances: ", len(valid_x))

Training instances:  298
Validation instances:  75


In [ ]:
"""Train the algorithm"""
# algorithm = Sgcn_Lstm(train_x, train_y, graph.AD, graph.AD2, graph.bias_mat_1, graph.bias_mat_2, lr = args.lr, epoach=args.epoch, batch_size=args.batch_size)
algorithm = Sgcn_Lstm(train_x, train_y, valid_x, valid_y, graph.AD, graph.AD2, lr = learning_rate, epoach=epoch, batch_size=batch_size)
model = algorithm.build()
history = algorithm.train()

In [ ]:
"""Test the model"""
y_pred = algorithm.prediction(valid_x)
y_pred = data_loader.sc2.inverse_transform(y_pred)
valid_y = data_loader.sc2.inverse_transform(valid_y) 

In [ ]:
"""Performance matric"""
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [ ]:
test_dev = abs(valid_y-y_pred)
mean_abs_dev = np.mean(test_dev)
mae = mean_absolute_error(valid_y, y_pred)
rms_dev = sqrt(mean_squared_error(y_pred, valid_y))
mse = mean_squared_error(valid_y,y_pred) 
mape = mean_absolute_percentage_error(valid_y, y_pred)

print('Mean absolute deviation:', mae)
print('RMS deviation:', rms_dev)
print('MSE:', mse)
print('MAPE: ', mape)